<a href="https://colab.research.google.com/github/Idriss-jedid/Automated-Contract-Photo-Verification/blob/master/Computer%20Vision%20/%20Car_Face_Classification_Using_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install timm


In [ ]:
import timm
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = '/content/drive/MyDrive/DATA/data.csv'
image_dir = '/content/drive/MyDrive/DATA/images/images'


In [ ]:
data = pd.read_csv(data_path)

In [ ]:
image_paths = [os.path.join(image_dir, img_name) for img_name in data['image']]
labels = data['class'].values

In [ ]:
# Split into train and test first
train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.20, random_state=42, stratify=labels
)

# Further split the train set into train and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(
    train_paths, train_labels, test_size=0.25, random_state=42, stratify=train_labels
)


In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
      image_path = self.image_paths[idx]
      image = Image.open(image_path).convert('RGB')
      label = self.labels[idx]
      if self.transform:
        image = self.transform(image)
      return image, label



In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# Create datasets
train_dataset = CustomImageDataset(train_paths, train_labels, transform)
val_dataset = CustomImageDataset(val_paths, val_labels, transform)
test_dataset = CustomImageDataset(test_paths, test_labels, transform)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
model = timm.create_model('vit_small_patch16_224', pretrained=True, num_classes=2)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
def train_one_epoch(model, criterion, optimizer, data_loader, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in data_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


In [ ]:
def validate_one_epoch(model, criterion, data_loader, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [ ]:
# Training and validation functions remain the same

num_epochs = 10
best_val_loss = float('inf')

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, criterion, optimizer, train_loader, device)
    val_loss, val_acc = validate_one_epoch(model, criterion, val_loader, device)

    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Train loss: {train_loss:.4f}, Train accuracy: {train_acc:.4f}')
    print(f'Val loss: {val_loss:.4f}, Val accuracy: {val_acc:.4f}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_vit_model2.pth')


/usr/local/lib/python3.10/dist-packages/PIL/JpegImagePlugin.py:846: UserWarning: Image appears to be a malformed MPO file, it will be interpreted as a base JPEG file
  warnings.warn(


Epoch 1/10
Train loss: 0.3643, Train accuracy: 0.8339
Val loss: 0.2122, Val accuracy: 0.9235
Epoch 2/10
Train loss: 0.1049, Train accuracy: 0.9642
Val loss: 0.2998, Val accuracy: 0.9104
Epoch 3/10
Train loss: 0.0636, Train accuracy: 0.9767
Val loss: 0.2251, Val accuracy: 0.9283
Epoch 4/10
Train loss: 0.0234, Train accuracy: 0.9908
Val loss: 0.2153, Val accuracy: 0.9446
Epoch 5/10
Train loss: 0.0126, Train accuracy: 0.9962
Val loss: 0.2223, Val accuracy: 0.9381
Epoch 6/10
Train loss: 0.0071, Train accuracy: 0.9978
Val loss: 0.2479, Val accuracy: 0.9430
Epoch 7/10
Train loss: 0.0036, Train accuracy: 0.9978
Val loss: 0.2755, Val accuracy: 0.9430
Epoch 8/10
Train loss: 0.0032, Train accuracy: 0.9989
Val loss: 0.2606, Val accuracy: 0.9430
Epoch 9/10
Train loss: 0.0030, Train accuracy: 0.9984
Val loss: 0.3149, Val accuracy: 0.9349
Epoch 10/10
Train loss: 0.0023, Train accuracy: 0.9989
Val loss: 0.2536, Val accuracy: 0.9430


In [ ]:
model.load_state_dict(torch.load('best_vit_model2.pth'))


<ipython-input-38-b993a9f96f57>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_vit_model2.pth'))


<All keys matched successfully>

In [ ]:
# Testing the model
def test_model(model, data_loader, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [ ]:
test_loss, test_acc = test_model(model, test_loader, device)
print(f'Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}')

Test loss: 0.1722, Test accuracy: 0.9301


In [ ]:
!pip install onnx
import torch
import timm
import onnx # Import the onnx module

# Instantiate the model
model = timm.create_model('vit_small_patch16_224', pretrained=True, num_classes=2)

# Load the state dictionary into the model
state_dict = torch.load("best_vit_model2.pth")
model.load_state_dict(state_dict)

model.eval()  # Now you can set the model to evaluation mode

# Create dummy input with the appropriate size
dummy_input = torch.randn(1, 3, 224, 224)

# Export the model
torch.onnx.export(model, dummy_input, "best_vit_model2.onnx", input_names=['input'], output_names=['class_scores'])


<ipython-input-41-7223db6d22c3>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("best_vit_model2.pth")
/usr/local/lib/python3.10/dist-packages/torc